# Structure of State Tensors

In this tutorial, we will go over the different types of states used in MotorNet, how they flow during simulation, how they are structured and what information they carry.

Let's start by importing what we need.


In [1]:

import os
import sys
import numpy as np
import tensorflow as tf

colab_env = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
colab_initialized = True if os.path.exists("MotorNet") else False

if colab_env and not colab_initialized:
  !git clone https://github.com/OlivierCodol/MotorNet
  sys.path.append('MotorNet')
  print("Running cell using COLAB initialization...")
elif colab_env and colab_initialized:
  print("Already initialized using COLAB initialization.")
else:
  paths = [p for p in sys.path if os.path.exists(p)]
  local_initialized = True if [p for p in paths if "motornet" in os.listdir(p)] else False
  if local_initialized:
    %load_ext autoreload
    %autoreload 2
    print("Already initialized using LOCAL initialization.")
  else:
    path = [p for p in paths if p.__contains__("tutorials")]
    if len(path) != 1:
      raise ValueError("Path to MotorNet could not be determined with certainty.")
    sys.path.append(os.path.dirname(path[:path.rfind('tutorials')]))
    %load_ext autoreload
    %autoreload 2
    print("Running cell using LOCAL initialization...")


import motornet as mn

print('All packages imported.')
print('tensorflow version: ' + tf.__version__)
print('numpy version: ' + np.__version__)


Already initialized using LOCAL initialization.
All packages imported.
tensorflow version: 2.9.1
numpy version: 1.22.4



# I. Types of States

States are the main mean of communication between MotorNet objects and so they convey a wide variety of information.

There are 7 states that will always be present regardless of the model being created.

- Joint state
- Cartesian state
- Muscle state
- Geometry state
- Proprioceptive feedback state
- Visual feedback state
- Excitation

Additionally, `Network` objects may have states associated with its functional units. For instance, Gated Recurrent Units (GRUs) or Long Short Term Memory (LSTM) units will have hidden activity that will need to get passed from timestep to timestep. Conversely, a simple multi-layer perceptron may not have any states associated with its computation.



# II. State flow at runtime

Below is an overview of where states are generated and updated in a MotorNet model, and how they flow from object to object.




<img src="img/states.png" alt="drawing" width="600"/>


From the above illustration, we can clearly see that `Muscle` objects generate muscle states, `Skeleton` objects generate joint and cartesian states, and `Plant` objects generate geometry states (though partially using `Skeleton` information). On the other hand, the `Network` object handles both proprioceptive and visual feedback states, and excitation states, on top of any additional network-related states that may be present if necessary for the employed network architecture.

Excitation states can be subject to noise, but the excitation states returned by the model are the noise-free excitation states. This is because, when looking at that particular state, most users will be after the network output (noiseless excitation) rather than muscle input (noisy excitation).



# III. Dimensionality of State tensors

## III. 1. Plant States
Let us create a plant, and then get initial states for a batch size of 7.
We use a pre-built plant with 4 muscle, and then add a fifth muscle. The plant's skeleton has 2 degrees of freedom and evolves in a 2D cartesian space.


In [2]:

plant = mn.plants.ReluPointMass24()

# adding a fifth muscle
plant.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[1, 0], [0, 0]], max_isometric_force=1)

states = plant.get_initial_state(batch_size=7)
labels = ["joint state", "cartesian state", "muscle state", "geometry state"]

for label, state in zip(labels, states):
  print(label + " shape: ", state.shape)


joint state shape:  (7, 4)
cartesian state shape:  (7, 4)
muscle state shape:  (7, 4, 5)
geometry state shape:  (7, 4, 5)


From the example above it is fairly easy to see that for all states, the first dimension always corresponds to the batch size. We can also see that for the muscle and geometry states, the last dimension is always the number of muscles.

The second dimension is the number of features of that state. For the joint state, this is the number of degrees of freedom times two (position and velocity). For the cartesian state, this is the dimensionality of the cartesian space (here 2D) times two (cartesian position of the effector's endpoint, cartesian velocity of the effector's endpoint).

For the geometry state, this will always be musculotendon length, musculotendon velocity, and the moment of the muscle considered for each joint. Because there are two degrees of freedom here for the skeleton (two joints), there are two moments. Additionally, this information can be obtained by checking the `Plant` object's `geometry_state_name` attribute.

For the muscle state, this depends on the muscle type being used. This information can be obtained by checking the `Muscle` object's `state_name` attribute. The `Muscle` object is directly accessible from the `Plant` object as demonstrated below.



In [3]:
features = plant.muscle.state_name
for n, feature in enumerate(features):
  print("feature " + str(n) + ": ", feature)


feature 0:  activation
feature 1:  muscle length
feature 2:  muscle velocity
feature 3:  force



And below, we fetch the geometry state names using the equivalent attribute for geometry at the `Plant` level.


In [4]:
features = plant.geometry_state_name
for n, feature in enumerate(features):
  print("feature " + str(n) + ": ", feature)


feature 0:  musculotendon length
feature 1:  musculotendon velocity
feature 2:  moment for joint 0
feature 3:  moment for joint 1


## III. 2. Network States

Let's now look at states at the `Network` level. First, we build a `Network` from the same plant as earlier. One difference is that we will include feedback delays to proprioception and vision.



In [5]:

plant = mn.plants.ReluPointMass24(proprioceptive_delay=0.03, visual_delay=0.09)

plant.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[1, 0], [0, 0]], max_isometric_force=1)  # adding a fifth muscle

network = mn.nets.layers.GRUNetwork(plant=plant, n_units=50, kernel_regularizer=10**-6, name='network')


Then we can print the state shapes from the `Network`.

In [6]:
shapes = network.output_size
names = network.output_names

for name, shape in zip(names, shapes):
  print(name + " shape: ", shape)

joint position shape:  (4,)
cartesian position shape:  (4,)
muscle state shape:  (4, 5)
geometry state shape:  (4, 5)
proprioceptive feedback shape:  (10, 3)
visual feedback shape:  (2, 9)
excitation shape:  (5,)
gru_hidden_0 shape:  (50,)


We can see that here, the batch size is omitted. This is because we are looking at symbolic tensors, and not eager tensors. This simply means that the first dimension is not displayed but in practice, at runtime, states will still have a first dimension representing batch size, and the shapes displayed here will be shifted one dimension to the right. We can also see that the `Plant` states are still available at this level.

Following on, we have the proprioceptive and visual feedback states. Those are backlogs of elements from the muscle and cartesian states, respectively. For proprioception, the features are the muscle length for all five muscles, followed by their velocity, leading to `2 * 5 = 10` features. For vision, this is the cartesian position. Since this is a 2D cartesian space, the first dimension contains two entries. For both feedback states, the last dimension is as large as the backlog, and therefore depends on the delay declared when building the `Plant`. Here, we specified a proprioceptive delay of 30 ms, and the timesteps are 10 ms, resulting in 3 timesteps. For vision, the delay is 90 ms, resulting in 9 timesteps.

The excitation shape depends on the number of muscles receiving descending drive from the `Newtork` object. Here, we have five muscles. Note the muscle type we use (ReLu muscles) requires only a scalar input, but if a muscle type requires more than one input, then the feature dimension of the excitation state would be multiplied by that.

Finally, if `Network` states are required, they will be placed at the end of this list. Here, we only have one layer of 50 GRUs, so we have one 50-features GRU hidden state.



# IV. Initial States from Task objects

Task objects can provide initial states from their `get_initial_state_layers` method. The list of states from this method has the same dimensionality, but the first dimension will be a `None` placeholder value for batch size.

In [7]:

task = mn.tasks.RandomTargetReach(network=network)
state0 = task.get_initial_state_layers()

for name, state in zip(names, state0):
  print(name + " shape: ", " " * (30 - len(name)), state.shape)


joint position shape:                   (None, 4)
cartesian position shape:               (None, 4)
muscle state shape:                     (None, 4, 5)
geometry state shape:                   (None, 4, 5)
proprioceptive feedback shape:          (None, 10, 3)
visual feedback shape:                  (None, 2, 9)
excitation shape:                       (None, 5)
gru_hidden_0 shape:                     (None, 50)


# V. States as returned from Model forward pass

When doing a forward pass on a model, the model will output states as a result. These states take a specific value and so are not symbolic tensors, but instanciated tensors. The main point to keep in mind is that an additional dimension will be inserted in second position (right after the batch dimension), which corresponds to timesteps. Here we are doing a 100 timesteps simulation, and therefore that dimension will be of size 100. All the remaining dimensions will be pushed off to the right accordingly.


In [8]:
# create a model
inputs = task.get_input_dict_layers()

rnn = tf.keras.layers.RNN(cell=network, return_sequences=True, name='RNN')
states_out = rnn(inputs, initial_state=state0)

model = mn.nets.MotorNetModel(inputs=[inputs, state0], outputs=states_out, name='model', task=task)
model.compile(optimizer=tf.optimizers.Adam(clipnorm=1.), loss=task.losses, loss_weights=task.loss_weights)

print("model compiled")

model compiled


In [9]:
# forward pass
[x, _, x0] = model.task.generate(n_timesteps=100, batch_size=7)
results = model([x, x0], training=False)
print("done")


done


In [10]:
for k, v in results.items():
  print(k, " " * (30 - len(k)), v.shape)

joint position                  (7, 100, 4)
cartesian position              (7, 100, 4)
muscle state                    (7, 100, 4, 5)
geometry state                  (7, 100, 4, 5)
proprioceptive feedback         (7, 100, 10, 3)
visual feedback                 (7, 100, 2, 9)
excitation                      (7, 100, 5)
gru_hidden_0                    (7, 100, 50)
